# High-level CNTK Example

In [1]:
import numpy as np
import os
import sys
import cntk
from cntk.layers import Convolution2D, MaxPooling, Dense, Dropout
from common.params import *
from common.utils import *

In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.6.3 |Anaconda, Inc.| (default, Nov 20 2017, 20:41:42) 
[GCC 7.2.0]
Numpy:  1.13.3
CNTK:  2.3
GPU:  ['Tesla K80']


In [3]:
def create_symbol():
    # Weight initialiser from uniform distribution
    # Activation (unless states) is None
    with cntk.layers.default_options(init = cntk.glorot_uniform(), activation = cntk.relu):
        x = Convolution2D(filter_shape=(3, 3), num_filters=50, pad=True)(features)
        x = Convolution2D(filter_shape=(3, 3), num_filters=50, pad=True)(x)
        x = MaxPooling((2, 2), strides=(2, 2), pad=False)(x)
        x = Dropout(0.25)(x)

        x = Convolution2D(filter_shape=(3, 3), num_filters=100, pad=True)(x)
        x = Convolution2D(filter_shape=(3, 3), num_filters=100, pad=True)(x)
        x = MaxPooling((2, 2), strides=(2, 2), pad=False)(x)
        x = Dropout(0.25)(x)    
        
        x = Dense(512)(x)
        x = Dropout(0.5)(x)
        x = Dense(N_CLASSES, activation=None)(x)
        return x

In [4]:
def init_model(m):
    # Loss (dense labels); check if support for sparse labels
    loss = cntk.cross_entropy_with_softmax(m, labels)
    # Momentum SGD
    # https://github.com/Microsoft/CNTK/blob/master/Manual/Manual_How_to_use_learners.ipynb
    # unit_gain=False: momentum_direction = momentum*old_momentum_direction + gradient
    # if unit_gain=True then ...(1-momentum)*gradient
    learner = cntk.momentum_sgd(m.parameters, 
                                lr=cntk.learning_rate_schedule(LR, cntk.UnitType.minibatch) , 
                                momentum=cntk.momentum_schedule(MOMENTUM),
                                unit_gain=False)
    return loss, learner

In [5]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True, one_hot=True)
# CNTK format
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 float32 float32
CPU times: user 849 ms, sys: 520 ms, total: 1.37 s
Wall time: 1.37 s


In [6]:
%%time
# Placeholders
features = cntk.input_variable((3, 32, 32), np.float32)
labels = cntk.input_variable(N_CLASSES, np.float32)
# Load symbol
sym = create_symbol()

CPU times: user 16.5 ms, sys: 36.1 ms, total: 52.6 ms
Wall time: 67.8 ms


In [7]:
%%time
loss, learner = init_model(sym)

CPU times: user 104 ms, sys: 248 ms, total: 352 ms
Wall time: 362 ms


In [8]:
%%time
# 171s
loss.train((x_train, y_train), 
           minibatch_size=BATCHSIZE, 
           max_epochs=EPOCHS,
           parameter_learners=[learner])

CPU times: user 1min 34s, sys: 1min 4s, total: 2min 38s
Wall time: 3min 41s


{'epoch_summaries': [{'loss': 1.82419796875, 'metric': 0.0, 'samples': 50000},
  {'loss': 1.34896015625, 'metric': 0.0, 'samples': 50000},
  {'loss': 1.127497734375, 'metric': 0.0, 'samples': 50000},
  {'loss': 0.989734296875, 'metric': 0.0, 'samples': 50000},
  {'loss': 0.876199375, 'metric': 0.0, 'samples': 50000},
  {'loss': 0.789623046875, 'metric': 0.0, 'samples': 50000},
  {'loss': 0.721123046875, 'metric': 0.0, 'samples': 50000},
  {'loss': 0.6605334375, 'metric': 0.0, 'samples': 50000},
  {'loss': 0.601108203125, 'metric': 0.0, 'samples': 50000},
  {'loss': 0.5586728515625, 'metric': 0.0, 'samples': 50000}],
 'updates': [{'loss': 1.824187802596831, 'metric': 0.0, 'samples': 49984},
  {'loss': 1.3488722641245199, 'metric': 0.0, 'samples': 49984},
  {'loss': 1.1274982213058178, 'metric': 0.0, 'samples': 49984},
  {'loss': 0.9896796662431978, 'metric': 0.0, 'samples': 49984},
  {'loss': 0.8761663888694382, 'metric': 0.0, 'samples': 49984},
  {'loss': 0.7896260397077065, 'metric': 

In [9]:
%%time
# Predict and then score accuracy
# (We don't need softmax -> monotonic function)
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = np.argmax(y_test[:n_samples], axis=-1)
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    predicted_label_probs = sym.eval({features : data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = np.argmax(predicted_label_probs, axis=-1)
    c += 1

CPU times: user 657 ms, sys: 596 ms, total: 1.25 s
Wall time: 1.84 s


In [10]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.763321314103
